## Memory - Context in Agents

In [1]:
!uv pip install langchain-core langgraph langgraph-checkpoint-postgres psycopg[binary,pool] openai


Using Python 3.13.7 environment at: /home/hp/Desktop/Workplace/CustomizeGPT/.venv
Audited 5 packages in 39ms


In [2]:
!uv pip install -U "psycopg[binary,pool]" langgraph langgraph-checkpoint-postgres

Using Python 3.13.7 environment at: /home/hp/Desktop/Workplace/CustomizeGPT/.venv
Resolved 35 packages in 2.44s                                        
⠙ Preparing packages... (0/13)                                                  
⠙ Preparing packages... (0/13)-------------     0 B/64.96 KiB           
⠹ Preparing packages... (10/13)------------ 16.00 KiB/64.96 KiB         
⠹ Preparing packages... (10/13)------------ 16.00 KiB/64.96 KiB         
⠹ Preparing packages... (10/13)------------ 32.00 KiB/64.96 KiB         
⠹ Preparing packages... (10/13)m----------- 40.72 KiB/64.96 KiB         
⠹ Preparing packages... (10/13)m----------- 40.72 KiB/64.96 KiB         
langgraph-sdk        ------------------------------ 40.72 KiB/64.96 KiB
⠹ Preparing packages... (10/13)------------     0 B/267.30 KiB          
langgraph-sdk        ------------------------------ 40.72 KiB/64.96 KiB
⠹ Preparing packages... (10/13)------------     0 B/267.30 KiB          
langgraph-sdk        ------------------

In [20]:
import os
SUPABASE_DB_URI = os.getenv("SUPABASE_DB_URI")
# print(SUPABASE_DB_URI)

In [4]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.postgres import PostgresSaver  

model = init_chat_model(model="gpt-4o-mini")

with PostgresSaver.from_conn_string(SUPABASE_DB_URI) as checkpointer:  
    checkpointer.setup()

    def call_model(state: MessagesState):
        response = model.invoke(state["messages"])
        return {"messages": response}

    builder = StateGraph(MessagesState)
    builder.add_node(call_model)
    builder.add_edge(START, "call_model")

    graph = builder.compile(checkpointer=checkpointer)  

    config = {
        "configurable": {
            "thread_id": "1"
        }
    }

    for chunk in graph.stream(
        {"messages": [{"role": "user", "content": "hi! I'm bob"}]},
        config,  
        stream_mode="values"
    ):
        chunk["messages"][-1].pretty_print()

    for chunk in graph.stream(
        {"messages": [{"role": "user", "content": "what's my name?"}]},
        config,  
        stream_mode="values"
    ):
        chunk["messages"][-1].pretty_print()

/home/hp/Desktop/Workplace/CustomizeGPT/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hi Bob! How can I assist you today?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob! How can I help you today, Bob?


In [10]:
# @app.get("/conversations/{conversation_id}")
async def get_conversation_history(conversation_id: str):
    """
    Retrieve the message history for a specific conversation (thread) from Postgres.
    """
    try:
        # Configuration for the checkpointer
        config = {"configurable": {"thread_id": conversation_id}}
        
        with PostgresSaver.from_conn_string(SUPABASE_DB_URI) as checkpointer:
            # Fetch the latest state checkpoint for this thread
            state = checkpointer.get_tuple(config)
            
            if not state:
                return {"messages": [], "message": "No history found for this thread."}

            # Extract the 'messages' channel from the state values
            # Note: This depends on your State definition (usually 'messages')
            checkpoint_values = state.checkpoint.get("channel_values", {})
            messages = checkpoint_values.get("messages", [])

            return {
                "thread_id": conversation_id,
                "messages": messages
            }
            
    except Exception as e:
        # raise HTTPException(status_code=500, detail=f"Failed to retrieve conversation: {str(e)}")
        print("error")

In [12]:
response = await get_conversation_history("1")
print(response)

{'thread_id': '1', 'messages': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='957052f1-447d-4a31-b0f6-a73355209844'), AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CpW78T5XSw26WxV4tcQXqZuQbYODd', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b454c-9f9a-79d3-af28-f50fff30a87f-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio':

/tmp/ipykernel_34692/2444764511.py:1: RuntimeWarning: coroutine 'get_conversation_history' was never awaited
  response = await get_conversation_history("1")


In [14]:
formatted_messages = [
    {"role": msg.type, "content": msg.content} for msg in response["messages"]
]

print(formatted_messages)

[{'role': 'human', 'content': "hi! I'm bob"}, {'role': 'ai', 'content': 'Hi Bob! How can I assist you today?'}, {'role': 'human', 'content': "what's my name?"}, {'role': 'ai', 'content': 'Your name is Bob! How can I help you today, Bob?'}]


## DPR + Reranking

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoder, DPRContextEncoderTokenizer
import torch, faiss, numpy as np
from sentence_transformers import CrossEncoder
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain.schema import Document

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base").to(device)

q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
q_model = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to(device)

cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

In [ ]:
class DPRVectorStore:
    def __init__(self, ctx_model, q_model, ctx_tokenizer, q_tokenizer):
        self.ctx_model = ctx_model
        self.q_model = q_model
        self.ctx_tokenizer = ctx_tokenizer
        self.q_tokenizer = q_tokenizer
        self.index = None
        self.passages = []
        self.embeddings = None

    def build_index(self, passages):
        self.passages = passages
        inputs = self.ctx_tokenizer(passages, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            self.embeddings = self.ctx_model(**inputs.to(device)).pooler_output.cpu().numpy()
        dim = self.embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dim)
        self.index.add(self.embeddings)

    def query(self, question, top_k=5):
        q_inputs = self.q_tokenizer(question, return_tensors="pt").to(device)
        with torch.no_grad():
            q_embed = self.q_model(**q_inputs).pooler_output.cpu().numpy()
        D, I = self.index.search(q_embed, top_k)
        results = [self.passages[i] for i in I[0]]
        return results

    def rerank(self, question, passages):
        pairs = [(question, p) for p in passages]
        scores = cross_encoder_model.predict(pairs)
        ranked = [p for _, p in sorted(zip(scores, passages), reverse=True)]
        return ranked, scores

    def save_index(self, path="dpr_index.faiss"):
        faiss.write_index(self.index, path)

    def load_index(self, path="dpr_index.faiss"):
        self.index = faiss.read_index(path)


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../data/Sales/Case Studies/Aura Health.pdf"
loader = PyPDFLoader(file_path)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

print("Number of chunks:", len(chunks))


Number of chunks: 19


In [ ]:
retriever = DPRVectorStore(ctx_model, q_model, ctx_tokenizer, q_tokenizer)
retriever.build_index([c.page_content for c in chunks])
print("Index built with", len(chunks), "chunks")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Index built with 19 chunks


In [ ]:
query = "What is Emotion Intelligence Integration?"

dpr_results = retriever.query(query, top_k=5)
reranked, scores = retriever.rerank(query, dpr_results)

print("DPR Top-k Results:")
for i, p in enumerate(dpr_results, 1):
    print(f"{i}. {p}")

print("\nCross-Encoder Reranked Results:")
for i, (p, s) in enumerate(zip(reranked, scores), 1):
    print(f"{i}. {p} (score: {s:.4f})")


DPR Top-k Results:
1. Emotion Detection Gap
Therapists and wellness coaches needed better tools to create, 
publish, and monetize their content, but the platform lacked an 
intuitive, scalable interface to support growing creator needs.
Creator Experience Limitations
Aura’s expansion into workplaces required robust infrastructure to 
handle enterprise-scale usage, real-time personalization, and
integration with collaboration tools all without sacriﬁcing
performance or privacy.
Scalability for Enterprise Use
2. Impact
Emotion AI delivered a 40% increase in content relevance, as users 
engaged more with mood-matched meditations, stories, and
therapy sessions.
Real-Time Personalization
Upgraded UI/UX and personalized content journeys led to a 35% 
uplift in daily active users and a 25% increase in session duration 
across platforms.
Boost in User Engagement
The creator tools resulted in a 60% increase in audio content 
ploads and a 3x growth in active wellness coaches contributing 
to the

In [90]:
from langgraph.prebuilt import ToolNode
from langchain.tools import tool

@tool
def retrieve_documents(query: str):
    """Retrieve relevant documents."""
    dpr_results = retriever.query(query, top_k=3)
    reranked, scores = retriever.rerank(query, dpr_results)

    return reranked

tools = [retrieve_documents]
tool_node = ToolNode(tools)

In [ ]:
EMAIL_SYSTEM_PROMPT = """
You are a professional email assistant for our company's sales team. Your role is to respond to customer inquiries using ONLY information from our knowledge base.

CRITICAL RULES:
1. You MUST respond in proper business email format with subject line, salutation, body, and signature
2. If the customer's question can be answered using the provided context, write a helpful, professional email response
3. If the information is NOT in the knowledge base (context shows "NO_RELEVANT_INFORMATION_FOUND"), respond with a polite email explaining this
4. Never invent information or use external knowledge
5. Maintain a professional, helpful tone in all communications
6. Format your response as a ready-to-send email
7. Always start the subject with "Re: " followed by the original subject or an appropriate title

EMAIL FORMAT:
Subject: Re: [Original Subject or Appropriate Title]

Dear [Customer Name],

[Professional email body acknowledging their query and providing information or explaining limitations]

[Clear next steps or contact information if needed]

Best regards,
[Sale Team]
[Strategisthub]
"""

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langgraph.graph import StateGraph, MessagesState
from langchain_core.messages import SystemMessage
from langgraph.graph import START, END
from langgraph.checkpoint.memory import MemorySaver

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

def call_model(state: MessagesState):
    response = model.invoke(
        [
            SystemMessage(
                        content=EMAIL_SYSTEM_PROMPT
                    )
        ]
        + state["messages"]
        )
    return {"messages": [response]}

def should_continue(state: MessagesState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

# 7. Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

# app = workflow.compile()
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [93]:
config = {"configurable": {"thread_id": "1"}}
response = app.invoke(
    {"messages": [{"role": "user", "content": query}]},
    config
)
print(response["messages"][-1].content)

Subject: Re: Inquiry About Emotion Intelligence Integration

Dear [Customer Name],

Thank you for your inquiry regarding Emotion Intelligence Integration. 

Emotion Intelligence Integration involves the development and deployment of AI models that detect emotional cues from user interactions across various platforms such as Slack, Zoom, and mobile/web applications. This technology enables dynamic content recommendations tailored to users' real-time mental states, enhancing user engagement and content relevance.

For instance, the implementation of this integration has led to a 40% increase in content relevance, as users engage more with mood-matched meditations, stories, and therapy sessions. Additionally, it has contributed to a significant boost in user engagement metrics.

If you have any further questions or need more detailed information, please feel free to reach out.

Best regards,  
Sales Team  
Strategisthub


In [94]:
query = "What is aura health?"
config = {"configurable": {"thread_id": "1"}}
response = app.invoke(
    {"messages": [{"role": "user", "content": query}]},
    config
)
print(response["messages"][-1].content)

Subject: Re: Inquiry About Aura Health

Dear [Customer Name],

Thank you for your inquiry about Aura Health.

Aura Health is a globally recognized leader in mental wellness, trusted by over 8 million users. It has been honored with several awards, including Apple’s Best of Apps and the Very Well Mind Online Therapy and Wellness Award. Aura provides a category-defining platform in digital wellness, combining the expertise of the world’s best coaches and therapists to deliver a deeply personalized self-care journey for each user.

The platform's vision is to create a global digital ecosystem for mental health, offering healing anytime and anywhere. A key feature of Aura is its Emotion AI, which reads user emotions from their interactions across various tools and recommends tailored content such as guided meditations, sleep stories, and cognitive therapy sessions.

If you have any further questions or need additional information, please feel free to reach out.

Best regards,  
Sales Team 

# Testing Graph

In [2]:
print("Testing...")

Testing...
